In [1]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("ML_Model").getOrCreate()
# spark.conf.set("spark.executor.memory", "8g")
# spark.conf.set("spark.executor.cores", "4")
# spark.conf.set("spark.driver.cores", "4")
# conf = SparkConf().set("spark.jars", "/Users/sz904/Downloads/postgresql-42.2.18.jar")
# sc = SparkContext( conf=conf)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 15:09:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
col_names = [
    'tcp_flags',
    'tcp_time_delta',
    'tcp_len',
    'mqtt_conack_flags',
    'mqtt_conack_flags_reserved',
    'mqtt_conack_flags_sp',
    'mqtt_conack_val',
    'mqtt_conflag_cleansess',
    'mqtt_conflag_passwd',
    'mqtt_conflag_qos',
    'mqtt_conflag_reserved',
    'mqtt_conflag_retain',
    'mqtt_conflag_uname',
    'mqtt_conflag_willflag',
    'mqtt_conflags',
    'mqtt_dupflag',
    'mqtt_hdrflags',
    'mqtt_kalive',
    'mqtt_len',
    'mqtt_msg',
    'mqtt_msgid',
    'mqtt_msgtype',
    'mqtt_proto_len',
    'mqtt_protoname',
    'mqtt_qos',
    'mqtt_retain',
    'mqtt_sub_qos',
    'mqtt_suback_qos',
    'mqtt_ver',
    'mqtt_willmsg',
    'mqtt_willmsg_len',
    'mqtt_willtopic',
    'mqtt_willtopic_len',
    'target',
    'dataset',
    'tcp_flags_decimal',
    'mqtt_conflags_decimal',
    'mqtt_hdrflags_decimal'
]

nominal_cols = ["mqtt_msg"]

corelated_cols_to_remove = ["mqtt_conack_flags",
                     "mqtt_conack_flags_reserved",
                     "mqtt_conack_flags_sp", 
                     "mqtt_conflag_qos", 
                     "mqtt_conflag_reserved", 
                     "mqtt_conflag_retain", 
                     "mqtt_conflag_willflag", 
                     "mqtt_willtopic", 
                     "mqtt_willtopic_len", 
                     "mqtt_sub_qos", 
                     "mqtt_suback_qos", 
                     "mqtt_willmsg",
                     "mqtt_willmsg_len"]



binary_cols = ["mqtt_conack_val", 
                  "mqtt_conflag_cleansess", 
                  "mqtt_conflag_passwd", 
                  "mqtt_conflag_uname", 
                  "mqtt_dupflag", 
                  "mqtt_proto_len", 
                  "mqtt_protoname", 
                  "mqtt_qos", 
                  "mqtt_retain", 
                  "mqtt_ver"]


continuous_cols = ["tcp_flags_decimal",
                     'tcp_time_delta',
                     'tcp_len',
                     'mqtt_kalive',
                     'mqtt_len',
                     'mqtt_msgid',
                     'mqtt_msgtype',
                     'mqtt_conflags_decimal', 
                     "mqtt_hdrflags_decimal", ]

In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

import numpy as np
from matplotlib import pyplot as plt


class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        # label_to_binary = udf(lambda name: 0.0 if name == 'normal' else 1.0)
        # output_df = dataset.withColumn('outcome', label_to_binary(col('class'))).drop("class")  
        # output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        # output_df = output_df.drop('difficulty')
        # return output_df

        def converter(name):
            if name == 'legitimate':
                return 0
            elif name == 'dos':
                return 1      
            elif name == 'malformed':
                return 2     
            elif name == 'flood':
                return 3
            elif name == 'bruteforce':
                return 4   
            elif name == 'slowite':
                return 5  
            else:
                print("out of range")
                return None
            
        label_to_binary = udf(lambda name: converter(name))
        output_df = dataset.withColumn('label', label_to_binary(col('target')))
        output_df = output_df.drop("target")  
        output_df = output_df.withColumn('label', col('label').cast(DoubleType()))
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in binary_cols + continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df



In [4]:
train_path = "./df_train_pandas.csv"
test_path = "./df_test_pandas.csv"
df_train = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)
print(df_train.count())
print(df_test.count())

231646
99290


In [5]:
df_train_updated = df_train
df_train_updated = df_train_updated.drop("tcp_flags")
df_train_updated = df_train_updated.drop("mqtt_conflags")
df_train_updated = df_train_updated.drop("mqtt_hdrflags")
df_train_updated = df_train_updated.drop("dataset")

df_test_updated = df_test
df_test_updated = df_test_updated.drop("tcp_flags")
df_test_updated = df_test_updated.drop("mqtt_conflags")
df_test_updated = df_test_updated.drop("mqtt_hdrflags")
df_test_updated = df_test_updated.drop("dataset")


df_train_updated = df_train_updated.drop("mqtt_msg")
df_test_updated = df_test_updated.drop("mqtt_msg")

df_train_updated = df_train_updated.drop("mqtt_conack_flags")
df_test_updated = df_test_updated.drop("mqtt_conack_flags")

df_train_updated = df_train_updated.drop("mqtt_protoname")
df_test_updated = df_test_updated.drop("mqtt_protoname")

In [7]:
outcomeCreate = OutcomeCreater()
stage_vector_assembler = VectorAssembler(inputCols=[column for column in df_train_updated.columns if column != 'target'], outputCol="vectorized_features")
stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')

for i in [column for column in df_train_updated.columns if column != 'target']:
    print(i)
print([column for column in df_train_updated.columns if column != 'target'])

tcp_time_delta
tcp_len
mqtt_conack_flags_reserved
mqtt_conack_flags_sp
mqtt_conack_val
mqtt_conflag_cleansess
mqtt_conflag_passwd
mqtt_conflag_qos
mqtt_conflag_reserved
mqtt_conflag_retain
mqtt_conflag_uname
mqtt_conflag_willflag
mqtt_dupflag
mqtt_kalive
mqtt_len
mqtt_msgid
mqtt_msgtype
mqtt_proto_len
mqtt_qos
mqtt_retain
mqtt_sub_qos
mqtt_suback_qos
mqtt_ver
mqtt_willmsg
mqtt_willmsg_len
mqtt_willtopic
mqtt_willtopic_len
tcp_flags_decimal
mqtt_conflags_decimal
mqtt_hdrflags_decimal
['tcp_time_delta', 'tcp_len', 'mqtt_conack_flags_reserved', 'mqtt_conack_flags_sp', 'mqtt_conack_val', 'mqtt_conflag_cleansess', 'mqtt_conflag_passwd', 'mqtt_conflag_qos', 'mqtt_conflag_reserved', 'mqtt_conflag_retain', 'mqtt_conflag_uname', 'mqtt_conflag_willflag', 'mqtt_dupflag', 'mqtt_kalive', 'mqtt_len', 'mqtt_msgid', 'mqtt_msgtype', 'mqtt_proto_len', 'mqtt_qos', 'mqtt_retain', 'mqtt_sub_qos', 'mqtt_suback_qos', 'mqtt_ver', 'mqtt_willmsg', 'mqtt_willmsg_len', 'mqtt_willtopic', 'mqtt_willtopic_len', 'tcp

In [51]:
buffer = outcomeCreate.transform(df_train_updated)
buffer = stage_vector_assembler.transform(buffer)
scaler_model = stage_scaler.fit(buffer)
buffer = scaler_model.transform(buffer)
buffer.printSchema()

root
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-- mqtt_msgtype: double (nullable = true)
 |-- mqtt_proto_len: double (nullable = true)
 |-- mqtt_qos: double (nullable = true)
 |-- mqtt_retain: double (nullable = true)
 |-- mqtt_sub_qos: double (nullable = true)
 |-- m

In [53]:
dropper = ColumnDropper(
    [column for column in buffer.columns if column != 'label' and column != 'features']
)
buffer = dropper.transform(buffer)
buffer.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [61]:
# logistic model
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')

# cross validation
lr_paramGrid = (ParamGridBuilder()
            .addGrid(lr.regParam, [0.01, 0.5, 2.0])
            .addGrid(lr.maxIter, [1, 5, 10])
            .build())

evaluator = MulticlassClassificationEvaluator(
    predictionCol='prediction', 
    labelCol='label', 
    metricName='accuracy'
)

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=5)


In [62]:
lr_model = lr_cv.fit(buffer)
res = lr_model.transform(buffer)

22/11/26 23:05:46 WARN CacheManager: Asked to cache already cached data.
22/11/26 23:05:46 WARN CacheManager: Asked to cache already cached data.


In [63]:
accuracy_test = (res.filter(res.label == res.prediction)
    .count() / float(res.count()))
print(f"Test Accuracy : {np.round(accuracy_test*100,2)}%")

Test Accuracy : 77.49%


In [27]:
df_train_updated.printSchema()

root
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-- mqtt_msgtype: double (nullable = true)
 |-- mqtt_proto_len: double (nullable = true)
 |-- mqtt_protoname: string (nullable = true)
 |-- mqtt_qos: double (nullable = true